In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layer
import numpy as np
import pandas as pd
import sklearn

# Local Bin
import src.lib.config as config

In [2]:
print(config.DATASET)

MDC


In [3]:
df = pd.read_csv(config.FM_MATRIX)
dfShape = df.shape
print(df.shape)

(2512822, 6)


In [4]:
inputs = keras.Input(shape=df.shape)


# TODO: identify timeSTEPS (delta for splitting by temporal resolution)

In [5]:
encoder = layer.LSTM(64, activation='tanh')(inputs)
encoder = layer.Dropout(0.2)(encoder)
encoder = layer.Dense(64, activation='relu')(encoder)
encoder = layer.Dropout(0.2)(encoder)
encoder = layer.Dense(200, activation='relu')(encoder)
encoder = layer.Dropout(0.2)(encoder)
encoder_out = layer.Dense(30, activation=None, name='encoder_out')(encoder)

In [6]:
from src.lstmae.clustering_layer import *

timesteps = int(dfShape[0] / 8)
n_features = dfShape[1]

In [8]:
clustering = ClusteringLayer(n_clusters=2, name='clustering', alpha=0.05)(encoder_out)
hidden = layer.RepeatVector(timesteps, name='Hidden')(encoder_out)
decoder = layer.Dense(200, activation='relu')(hidden)
decoder = layer.Dense(64, activation='relu')(decoder)
decoder = layer.LSTM(64, activation='tanh', return_sequences=True)(decoder)
output = layer.TimeDistributed(layer.Dense(n_features), name='decoder_out')(decoder)

<bound method Layer.set_weights of <src.lstmae.ClusteringLayer.ClusteringLayer object at 0x7fc738262f10>>


In [9]:
kmeans = KMeans(n_clusters=6, n_init=100) 
encoder_model = Model(inputs=inputs, outputs=encoder_out)
kmeans.fit(encoder_model.predict(dfShape))    
model = Model(inputs=inputs, outputs=[clustering, output])
clustering_model = Model(inputs=inputs, outputs=clustering)

NameError: name 'KMeans' is not defined